## ID3决策树

### 1. 导入相应的包

In [1]:
import pandas as pd
import numpy as np
import copy
from collections import deque

### 2. 相关的类

In [2]:
# 决策树的节点类，包含节点信息
class Node:
    def __init__(self):
        
        # 进行分割的特征名称
        self.feature_name = ""
        # 信息增益
        self.information_gain = 0
        # 儿子节点
        self.son_dict = dict()
        # 如果是叶节点，分类
        self.species = None
        # 样本个数
        self.samples = 0
    
    def print(self):
        """打印节点信息"""
        print("-------------------------")
        print("feature name: " + self.feature_name)
        print("information gain: %.2f" % self.information_gain)
        print("samples: " + str(self.samples))
        if self.species:
            print("leaf node, the species is: " + str(self.species))
        print("-------------------------")


class DecisionTreeID3:
    """决策树类，里面的数据全是 pandas 的 DataFrame"""
    
    def __init__(self, data_x=None, label=None, name=None):
        self.epsilon = 0.1
        self.tree_root = Node()
        self.train_data_x = data
        self.species_name = name
        self.label = label
        self.tree_dict = {}

    def create_tree(self, node, data_x, y):
        """递归建树"""
        
        # 每个标签对应的信息增益 (information gain)
        information_gain_list = self.cacu_information_gain(data_x, y)
        # 最大的信息增益 
        max_ig = information_gain_list.max()
        # 对应的特征
        argmax_feature = information_gain_list.idxmax()
        
        node.information_gain = max_ig
        node.feature_name = argmax_feature
        node.samples = data_x.shape[0]
        
        # 叶节点
        if max_ig < self.epsilon:
            classify_counts = y.value_counts()
            node.species = classify_counts.idxmax()
            return

        # 最大的特征对应不同取值
        items = data_x[argmax_feature].unique()
        for item in items:
            son = Node()
            # 递归
            self.create_tree(son,
                             data_x[data_x[argmax_feature] == item].drop(columns=argmax_feature),
                             y[data_x[argmax_feature] == item])
            node.son_dict[item] = son

    def fit(self):
        """根据建立模型， 也就是递归建树"""
        if self.tree_root is None:
            self.tree_root = Node()
        self.create_tree(self.tree_root, self.train_data_x, self.label)
    

    def cacu_information_gain(self, data_x, y):
        """计算不同特征信息增益"""
        feature_names = data_x.columns
        cond_entropy = []
        entropy_total = self.cacu_entropy(y)
        D = data_x.shape[0]
        
        # 遍历特征
        for name in feature_names:
            entropy = 0
            ser = data[name]
            counts = ser.value_counts()
            
            # 特征下不同的取值
            for item in counts.index:
                d = counts[item]
                entropy += d / D * self.cacu_entropy(y[ser == item])
            cond_entropy.append(entropy)
        return pd.Series(entropy_total - cond_entropy, index=feature_names)
    
    def cacu_entropy(self, data):
        """计算熵(entropy)"""
        entropy = 0.0
        C = data.count()
        counts = data.value_counts()
        for c in counts:
            if c != 0:
                entropy += - (c / C) * np.log2(c / C)
        return entropy

    def classify(self, X):
        """对数据分类"""
        node = self.tree_root
        while not (node.specie is None):
            feature = node.feature_name
            value = X[feature]
            node = node.son_dict[value]
        return node.species

    
    def __get_tree_dict(self, node):
        """私有方法，用字典的方式返回决策树: {特征：{取值：子树}}"""
        best_feature = node.feature_name
        if not (node.species is None):
            return node.species
        tree_dict = {best_feature: {}}
        for key, value in node.son_dict.items():
            tree_dict[best_feature][key] = self.__get_tree_dict(value)
        return tree_dict
    
    def get_tree_dict(self):
        """返回字典"""
        if not bool(self.tree_dict) or self.tree_dict is None:
            self.tree_dict = self.__get_tree_dict(self.tree_root)            
        return  copy.deepcopy(self.tree_dict)

### 3.测试 

In [3]:
# 预处理
data = pd.read_csv('play_tennis.csv')
data.drop(columns='day', inplace=True)
y = data['play']
data.drop(columns='play', inplace=True)

# 模型
model = DecisionTreeID3(data_x=data, label=y, name="play")
model.fit()

# 打印
tree_dict = model.get_tree_dict()
tree_dict

{'outlook': {'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'wind': {'Weak': 'Yes', 'Strong': 'No'}}}}